# Elevation torque spike data analysis

Craig Lage - 14-apr-25

In [ ]:
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.tmaUtils import TMAEventMaker
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from astropy.time import Time, TimeDelta

In [ ]:
client = makeEfdClient()
butler = butlerUtils.makeDefaultButler(instrument='LSSTCam')
#butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 'LSSTCam/calib/unbounded'])

In [ ]:
expId = 2025042100293
dataId = {'exposure': expId, 'detector': 90, 'instrument': 'LSSTCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
rotData = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=expRecord)
pos = rotData['actualPosition'].values.mean()
print(f"For {expId}, mean rotator position was {pos:.2f} degrees")

In [ ]:
elevationData = getEfdData(client=client, topic="lsst.sal.MTMount.elevation", expRecord=expRecord)
rotPos = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=expRecord)
rotTorque = getEfdData(client=client, topic="lsst.sal.MTRotator.motors", expRecord=expRecord)
hexPos = getEfdData(client=client, topic="lsst.sal.MTHexapod.application", expRecord=expRecord)
camHexPos = hexPos[hexPos['salIndex'] == 1]
m2HexPos = hexPos[hexPos['salIndex'] == 2]
hexTorque = getEfdData(client=client, topic="lsst.sal.MTHexapod.electrical", expRecord=expRecord)
camHexTorque = hexTorque[hexTorque['salIndex'] == 1]
m2HexTorque = hexTorque[hexTorque['salIndex'] == 2]
M1M3_forces = getEfdData(client=client, topic="lsst.sal.MTM1M3.forceActuatorData", expRecord=expRecord)
M2_axial_forces = getEfdData(client=client, topic="lsst.sal.MTM2.axialForce", expRecord=expRecord)
M2_tangent_forces = getEfdData(client=client, topic="lsst.sal.MTM2.tangentForce", expRecord=expRecord)

In [ ]:
fig = plt.figure(figsize=(10,10))
axs = fig.subplots(6,2, sharex=True)
plt.subplots_adjust(hspace=0.0)
plt.suptitle(f"Elevation torque glitches {expId}", fontsize=24)
axs[0][0].set_title("Elevation", y=1.0, pad=-15)
el = elevationData['actualPosition']
el.plot(ax = axs[0][0])
axs[1][0].set_title("Elevation Torque", y=1.0, pad=-15)
elTorque = elevationData['actualTorque'] - elevationData['actualTorque'].median()
elTorque.plot(ax = axs[1][0])
axs[2][0].set_title("Rotator Position", y=1.0, pad=-15)
rot = rotPos['actualPosition'] - rotPos['actualPosition'].median()
rot.plot(ax = axs[2][0])
axs[3][0].set_title("Rotator Torques", y=1.0, pad=-15)
rotT0 = rotTorque['torque0'] - rotTorque['torque0'].median()
rotT0.plot(ax = axs[3][0])
rotT1 = rotTorque['torque1'] - rotTorque['torque1'].median()
rotT1.plot(ax = axs[3][0])
axs[4][0].set_title("CamHexPos", y=1.0, pad=-15)
for i in range(6):
    cam = camHexPos[f'position{i}'] - camHexPos[f'position{i}'].median()
    cam.plot(ax = axs[4][0])
axs[5][0].set_title("CamHexTorque", y=1.0, pad=-15)
for i in range(6):
    cam = camHexTorque[f'motorCurrent{i}'] - camHexTorque[f'motorCurrent{i}'].median()
    cam.plot(ax = axs[5][0])
axs[0][1].set_title("Elevation Torque", y=1.0, pad=-15)
elTorque = elevationData['actualTorque'] - elevationData['actualTorque'].median()
elTorque.plot(ax = axs[0][1])
axs[1][1].set_title("M2HexPos", y=1.0, pad=-15)
for i in range(6):
    m2 = m2HexPos[f'position{i}'] - m2HexPos[f'position{i}'].median()
    m2.plot(ax = axs[1][1])
axs[2][1].set_title("M2HexTorque", y=1.0, pad=-15)
for i in range(6):
    m2 = m2HexTorque[f'motorCurrent{i}'] - m2HexTorque[f'motorCurrent{i}'].median()
    m2.plot(ax = axs[2][1])
axs[3][1].set_title("M1M3 Actuator Forces", y=1.0, pad=-15)
for i in range(156):
    m1m3 = M1M3_forces[f'zForce{i}'] - M1M3_forces[f'zForce{i}'].median()
    m1m3.plot(ax = axs[3][1])
axs[4][1].set_title("M2 Tangent Forces", y=1.0, pad=-15)
for i in range(6):
    m2f = M2_tangent_forces[f'measured{i}'] - M2_tangent_forces[f'measured{i}'].median()
    m2f.plot(ax = axs[4][1])
axs[5][1].set_title("M2 Axial Forces", y=1.0, pad=-15)
for i in range(6):
    m2f = M2_axial_forces[f'measured{i}'] - M2_axial_forces[f'measured{i}'].median()
    m2f.plot(ax = axs[5][1])

plt.savefig(f"/home/cslage/DATA/Elevation_Torque_Glitches_{expId}_14Apr25.png")

In [ ]:
expIds = [2025041300473, 2025041300481, 2025041300482]
for expId in expIds:
    print(f"expId = {expId}")
    dataId = {'exposure': expId, 'detector': 90, 'instrument': 'LSSTCam'}
    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    elevationData = getEfdData(client=client, topic="lsst.sal.MTMount.elevation", expRecord=expRecord)
    
    elTorque = elevationData['actualTorque'] - elevationData['actualTorque'].median()
    lastTimestamp = 1E12
    for i, value in enumerate(elTorque.values):
        if abs(value) > 500.0:
            thisTimestamp = Time(elevationData['timestamp'].iloc[i], format='unix_tai', scale='utc')
            #print(thisTimestamp.isot)
            if abs(thisTimestamp.unix_tai - lastTimestamp) > 1.0:
                print(Time(elevationData['timestamp'].iloc[i], format='unix_tai', scale='utc').isot)
                lastTimestamp = thisTimestamp.unix_tai

In [ ]:
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs=20250415)

In [ ]:
events[199]

In [ ]:
elevationData = getEfdData(client=client, topic="lsst.sal.MTMount.elevation", 
                          event=events[199])

targetData = getEfdData(client=client, topic="lsst.sal.MTMount.command_trackTarget", 
                          event=events[199])

# Calculate the derivative of the commanded position values
el = targetData['elevation'].values
times = targetData['taiTime'].values
times = times - times [0]

dev = np.zeros([len(el) - 1])
for i in range(1, len(el)):
    dev[i-1] = (el[i] - el[i-1]) / (times[i] - times[i-1])
devTimes = times[0:-1]
print(len(dev), len(devTimes))  

fig, axs = plt.subplots(2,1,figsize = (5,5))
plt.subplots_adjust(hspace=0.7)
plt.suptitle(f"Elevation glitches 20250415 TMA event 199")
axs[0].set_title("Elevation torque")
elevationData['actualTorque'].plot(ax = axs[0])                         
axs[1].set_title("Derivative of commanded postions")
axs[1].plot(devTimes, dev)
plt.savefig(f"/home/cslage/DATA/Elevation_Torque_Glitches_TMAEvent_199_16Apr25.png")

In [ ]:
expId = 2025042400197
dataId = {'exposure': expId, 'detector': 94, 'instrument': 'LSSTCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
rotData = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=expRecord)
pos = rotData['actualPosition'].values.mean()
print(f"For {expId}, mean rotator position was {pos:.2f} degrees")

In [ ]:
elevationData = getEfdData(client=client, topic="lsst.sal.MTMount.elevation", 
                          expRecord=expRecord)

targetData = getEfdData(client=client, topic="lsst.sal.MTMount.command_trackTarget", 
                          expRecord=expRecord)
# Calculate the derivative of the commanded position values
el = targetData['elevation'].values
times = targetData['taiTime'].values
times = times - times [0]

dev = np.zeros([len(el) - 1])
for i in range(1, len(el)):
    dev[i-1] = (el[i] - el[i-1]) / (times[i] - times[i-1])
devTimes = times[0:-1]
print(len(dev), len(devTimes))  

fig, axs = plt.subplots(3,1,figsize = (8,10))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Elevation glitches {expId}", fontsize=18)
axs[0].set_title("Elevation torque")
elevationData['actualTorque'].plot(ax = axs[0])                         
axs[1].set_title("Derivative of commanded postions")
axs[1].plot(devTimes, dev)
axs[2].set_title("Commanded positions with linear fit subtracted")
cfit = np.polyfit(times, el, 1)
elMinusFit = (el - cfit[0] * times - cfit[1]) * 3600.0
axs[2].plot(times, elMinusFit, marker='x')
axs[2].set_xlabel("Time(seconds)")
axs[2].set_ylabel("Arcseconds")
plt.savefig(f"/home/cslage/DATA/Elevation_Torque_Glitches_Blowup_{expId}.png")